## Importations des Bibliotheques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

In [8]:
data = pd.read_csv("Dataset_Abandon_Scolaire.csv")
data.head(10)

,Age,Sexe,Taux_presence,Nombre_retards,Note_moyenne,Situation_familiale,Abandon
0,21,Femme,89.585023,2,15.826896,Enfants à charge,0
1,20,Homme,70.713068,3,12.829358,Célibataire,0
2,22,Femme,82.122963,1,13.086319,Enfants à charge,0
3,24,Homme,87.408859,3,14.992032,Célibataire,0
4,20,Homme,86.470676,3,12.671649,Marié,0
5,20,Femme,100.000000,0,15.343097,Marié,0
6,24,Homme,87.798400,1,11.869304,Célibataire,0
7,22,Femme,76.967563,3,13.868003,Célibataire,0
8,20,Femme,84.236286,2,12.621463,Enfants à charge,0
9,22,Homme,70.790029,4,10.941972,Marié,0


**Interpretation**

In [9]:
data.shape
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  2000 non-null   int64  
 1   Sexe                 2000 non-null   object 
 2   Taux_presence        2000 non-null   float64
 3   Nombre_retards       2000 non-null   int64  
 4   Note_moyenne         2000 non-null   float64
 5   Situation_familiale  2000 non-null   object 
 6   Abandon              2000 non-null   int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 109.5+ KB


**Interpretation**

In [ ]:
# # Générer le profil
# profile = ProfileReport(data, title="Analysis of abandon school", explorative=True, minimal=True)
# profile.to_file("Analysis of abandon school.html")

**Interpretation**